# SPU over local network

This example spins up a 2-party SPU network. Both parties listen on 0.0.0.0, and will connect to each other over the local network instead of through the loopback interface (localhost).


First, start an example 2-party SecretFlow network in simulation mode:


In [1]:
import secretflow
from secretnote.compat.secretflow.device.driver import SFConfigSimulationFullyManaged

secretflow.shutdown()

secretflow_config = SFConfigSimulationFullyManaged(parties=["alice", "bob"])
secretflow.init(**secretflow_config.dict())

alice = secretflow.PYU("alice")
bob = secretflow.PYU("bob")

2023-09-18 07:46:13,277	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Figure out what the local IP address is for this machine. Depending on your network configuration, there may be several applicable addresses (e.g. one on Wireless and one on Ethernet). We will be using [`netifaces`][netifaces] to find the first interface that is up and has an `AF_INET` address assigned to it:

[netifaces]: https://pypi.org/project/netifaces/


In [2]:
import netifaces

In [3]:
for iface in netifaces.interfaces():
    if iface.startswith("lo"):
        # skip loopback interface
        continue
    if iface.startswith("vbox"):
        # skip virtualbox interfaces
        continue
    details = netifaces.ifaddresses(iface)
    if netifaces.AF_INET not in details:
        # skip interfaces without an IP address
        continue
    inet_addr = details[netifaces.AF_INET][0]["addr"]
    break
else:
    raise RuntimeError("No suitable network interface found")

print(f"Using interface {iface} with address {inet_addr}")

Using interface en0 with address 30.231.120.44


Find two ports that are available on this machine. We will be using [`portpicker`][portpicker]:

[portpicker]: https://pypi.org/project/portpicker/


In [4]:
import portpicker

In [5]:
port_for_alice = portpicker.pick_unused_port()
port_for_bob = portpicker.pick_unused_port()

alice_bind_to = f"127.0.0.1:{port_for_alice}"
bob_bind_to = f"127.0.0.1:{port_for_bob}"

connect_to_alice_via = f"{inet_addr}:{port_for_alice}"
connect_to_bob_via = f"{inet_addr}:{port_for_bob}"

print(
    f"""Network layout:
alice: {alice_bind_to} -> {connect_to_alice_via}
bob:   {bob_bind_to} -> {connect_to_bob_via}"""
)

Network layout:
alice: 127.0.0.1:54238 -> 30.231.120.44:54238
bob:   127.0.0.1:54239 -> 30.231.120.44:54239


Spin up the SPU network:


In [6]:
from secretnote.compat.spu import (
    SPUClusterDef,
    SPUConfig,
    SPURuntimeConfig,
    SPUProtocolKind,
    SPUFieldType,
    SPUNode,
)

mpc_config = SPUConfig(
    cluster_def=SPUClusterDef(
        nodes=[
            SPUNode(
                party="alice",
                listen_addr=alice_bind_to,
                address=connect_to_alice_via,
            ),
            SPUNode(
                party="bob",
                listen_addr=bob_bind_to,
                address=connect_to_bob_via,
            ),
        ],
        runtime_config=SPURuntimeConfig(
            protocol=SPUProtocolKind.SEMI2K,
            field=SPUFieldType.FM128,
        ),
    ),
)

mpc = secretflow.SPU(**mpc_config.dict())
mpc.init()

Run an example JAX program on the SPU network:


In [7]:
import jax.numpy as jnp

In [8]:
def dot(x, y):
    return jnp.dot(x, y)

In [9]:
product = mpc(dot)(
    secretflow.to(alice, jnp.arange(10)).to(mpc),
    secretflow.to(bob, jnp.arange(10, 20)).to(mpc),
)
secretflow.reveal(product)

INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'
INFO:jax._src.xla_bridge:Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.
(_run pid=15138) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=15138) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=15138) INFO:jax._src.xla_bridge:Unable to initialize back

(SPURuntime pid=15173) 2023-09-18 07:46:16.382 [info] [default_brpc_retry_policy.cc:DoRetry:52] socket error, sleep=1000000us and retry


(_run pid=15169) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=15169) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=15169) INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'
(_run pid=15169) INFO:jax._src.xla_bridge:Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.


(SPURuntime pid=15173) 2023-09-18 07:46:17.382 [info] [default_brpc_retry_policy.cc:LogHttpDetail:33] cntl ErrorCode '64', http status code '200', response header '', error msg '[E61]Fail to connect Socket{id=0 addr=30.231.120.44:54238} (0x0x1380f8000): Connection refused [R1][E64]Not connected to 30.231.120.44:54238 yet, server_id=0'
(SPURuntime pid=15173) 2023-09-18 07:46:17.383 [info] [default_brpc_retry_policy.cc:DoRetry:75] aggressive retry, sleep=1000000us and retry
(SPURuntime pid=15173) 2023-09-18 07:46:18.383 [info] [default_brpc_retry_policy.cc:LogHttpDetail:33] cntl ErrorCode '64', http status code '200', response header '', error msg '[E61]Fail to connect Socket{id=0 addr=30.231.120.44:54238} (0x0x1380f8000): Connection refused [R1][E64]Not connected to 30.231.120.44:54238 yet, server_id=0 [R2][E64]Not connected to 30.231.120.44:54238 yet, server_id=0'
(SPURuntime pid=15173) 2023-09-18 07:46:18.383 [info] [default_brpc_retry_policy.cc:DoRetry:75] aggressive retry, sleep=100

array(735, dtype=int32)

Stop everything:


In [10]:
mpc.shutdown()
secretflow.shutdown()